In [ ]:
# ===========================
# Import necessary libraries
# ===========================
import os
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Conv1D, MaxPool1D, ReLU, Dropout, Softmax, concatenate, Conv2D
from keras.layers import LSTM, Permute, Reshape, ZeroPadding2D, Activation
from keras.utils.vis_utils import plot_model

# ===========================
# Define a CLDNN-like model
# ===========================
def CLDNNLikeModel(weights=None,
                   input_shape1=[2,1024],
                   classes=26,
                   **kwargs):
    """
    CLDNN-like model combining CNN + LSTM + Dense layers for modulation classification.
    Supports optional loading of pre-trained weights.
    """

    # Check if weights path exists if provided
    if weights is not None and not (os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), '
                         'or the path to the weights file to be loaded.')

    dr = 0.5  # Dropout rate to reduce overfitting

    # Input layer: shape = (channels, features) with an extra dimension for Conv2D
    input_x = Input(shape=(1, 2, 1024))

    # Zero-padding to maintain dimensions after convolution
    input_x_padding = ZeroPadding2D((0, 2), data_format="channels_first")(input_x)

    # -------------------
    # Convolutional layers
    # -------------------
    layer11 = Conv2D(50, (1, 8), padding='valid', activation="relu", name="conv11",
                     kernel_initializer="glorot_uniform", data_format="channels_first")(input_x_padding)
    layer11 = Dropout(dr)(layer11)  # Dropout for regularization

    layer11_padding = ZeroPadding2D((0, 2), data_format="channels_first")(layer11)
    layer12 = Conv2D(50, (1, 8), padding="valid", activation="relu", name="conv12",
                     kernel_initializer="glorot_uniform", data_format="channels_first")(layer11_padding)
    layer12 = Dropout(dr)(layer12)

    layer12 = ZeroPadding2D((0, 2), data_format="channels_first")(layer12)
    layer13 = Conv2D(50, (1, 8), padding='valid', activation="relu", name="conv13",
                     kernel_initializer="glorot_uniform", data_format="channels_first")(layer12)
    layer13 = Dropout(dr)(layer13)

    # -------------------
    # Concatenate first and last conv layers for richer features
    # -------------------
    concat = keras.layers.concatenate([layer11, layer13])
    concat_size = list(np.shape(concat))
    input_dim = int(concat_size[-1] * concat_size[-2])  # Flatten feature dimensions
    timesteps = int(concat_size[-3])  # Number of timesteps for LSTM

    # Reshape to (samples, timesteps, input_dim) for LSTM
    concat = Reshape((timesteps, input_dim))(concat)

    # -------------------
    # LSTM layer for temporal feature extraction
    # -------------------
    lstm_out = LSTM(50, input_dim=input_dim, input_length=timesteps)(concat)

    # -------------------
    # Dense layers for classification
    # -------------------
    layer_dense1 = Dense(256, activation='relu', kernel_initializer='he_normal', name="dense1")(lstm_out)
    layer_dropout = Dropout(dr)(layer_dense1)
    layer_dense2 = Dense(26, kernel_initializer='he_normal', name="dense2")(layer_dropout)

    # Softmax activation to output probabilities for each class
    layer_softmax = Activation('softmax')(layer_dense2)
    output = Reshape([26])(layer_softmax)  # Ensure output shape matches number of classes

    # Build the model
    model = Model(inputs=input_x, outputs=output)

    # Load pre-trained weights if provided
    if weights is not None:
        model.load_weights(weights)

    return model


# ===========================
# Main execution: compile and inspect model
# ===========================
import keras
if __name__ == '__main__':
    # Instantiate the CLDNN-like model
    model = CLDNNLikeModel(None, input_shape=(2,1024), classes=24)

    # Compile the model with categorical crossentropy loss and Adam optimizer
    adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999,
                                 epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

    # Print detailed information about the model
    print('Model layers:', model.layers)        # List of all layers
    print('Model config:', model.get_config())  # Configuration of the model
    print('Model summary:', model.summary())   # Summary of layers, output shapes, and parameters

In [ ]:
import matplotlib
matplotlib.use('TkAgg')  # Use TkAgg backend for interactive plotting
import matplotlib.pyplot as plt
import numpy as np
import pickle
import csv
import itertools
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error, mean_absolute_error, r2_score

# ===========================
# Function: Plot confusion matrix
# ===========================
def plot_confusion_matrix(cm, title='', cmap=plt.get_cmap("Blues"), labels=[], save_filename=None):
    """
    Visualizes a confusion matrix with class labels and optional saving as PDF.
    """
    plt.figure(figsize=(10, 7))
    plt.imshow(cm*100, interpolation='nearest', cmap=cmap)  # Multiply by 100 to show in %
    plt.title(title, fontsize=10)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=90, size=12)
    plt.yticks(tick_marks, labels, size=12)

    # Annotate each cell with the value
    for i in range(len(tick_marks)):
        for j in range(len(tick_marks)):
            text_color = 'darkorange' if i == j else 'black'
            text = plt.text(j, i, int(np.around(cm[i,j]*100)), ha="center", va="center", fontsize=10, color=text_color)

    plt.tight_layout()
    plt.ylabel('True label', fontdict={'size':16})
    plt.xlabel('Predicted label', fontdict={'size':16})

    # Save figure if filename provided
    if save_filename is not None:
        plt.savefig(save_filename, format='pdf', dpi=1200, bbox_inches='tight')
    plt.close()

# ===========================
# Function: Calculate confusion matrix
# ===========================
def calculate_confusion_matrix(Y, Y_hat, classes):
    """
    Computes the normalized confusion matrix and counts of correct and incorrect predictions.
    """
    n_classes = len(classes)
    conf = np.zeros([n_classes, n_classes])
    confnorm = np.zeros([n_classes, n_classes])

    for k in range(Y.shape[0]):
        i = list(Y[k,:]).index(1)           # True class index
        j = int(np.argmax(Y_hat[k,:]))      # Predicted class index
        conf[i,j] += 1

    # Normalize each row to sum to 1
    for i in range(n_classes):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])

    right = np.sum(np.diag(conf))           # Correct predictions
    wrong = np.sum(conf) - right            # Incorrect predictions
    return confnorm, right, wrong

# ===========================
# Function: Calculate per-class accuracy from confusion matrix
# ===========================
def calculate_acc_at1snr_from_cm(cm):
    """
    Returns the recognition accuracy per class from the confusion matrix.
    """
    return np.round(np.diag(cm) / np.sum(cm, axis=1), 3)

# ===========================
# Function: Compute standard metrics
# ===========================
def calculate_metrics(Y, Y_hat):
    """
    Computes accuracy, precision, recall, F1-score, MSE, MAE, and R2 score.
    """
    Y_true = np.argmax(Y, axis=1)
    Y_pred = np.argmax(Y_hat, axis=1)

    accuracy = accuracy_score(Y_true, Y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(Y_true, Y_pred, average='weighted')

    mse = mean_squared_error(Y_true, Y_pred)
    mae = mean_absolute_error(Y_true, Y_pred)
    r2 = r2_score(Y_true, Y_pred)

    return accuracy, precision, recall, f1

# ===========================
# Function: Calculate metrics per SNR and optionally plot
# ===========================
def calculate_acc_cm_each_snr(Y, Y_hat, Z, classes=None, save_figure=True, min_snr=0):
    """
    Computes accuracy, precision, recall, and F1-score for each SNR level.
    Optionally generates plots of metrics vs SNR.
    """
    Z_array = Z[:, 0]  # Extract SNR values
    snrs = sorted(list(set(Z_array)))  # Unique SNR levels
    acc_mod_snr = np.zeros((len(classes), len(snrs)))

    metrics = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': []
    }

    # Loop over each SNR and compute metrics
    for snr in snrs:
        Y_snr = Y[np.where(Z_array == snr)]
        Y_hat_snr = Y_hat[np.where(Z_array == snr)]

        accuracy, precision, recall, f1 = calculate_metrics(Y_snr, Y_hat_snr)
        metrics['accuracy'].append(accuracy)
        metrics['precision'].append(precision)
        metrics['recall'].append(recall)
        metrics['f1'].append(f1)

    # Plot metrics vs SNR
    for metric, values in metrics.items():
        plt.figure(figsize=(8, 6))
        plt.plot(snrs, values, label=metric)
        for x, y in zip(snrs, values):
            plt.text(x, y, round(y, 3), ha='center', va='bottom', fontsize=8)
        plt.xlabel("Signal to Noise Ratio (dB)")
        plt.ylabel(metric.capitalize())
        plt.title(f"{metric.capitalize()} vs SNR")
        plt.legend()
        plt.grid()
        plt.savefig(f'figure/{metric}_vs_snr.pdf', format='pdf', dpi=1200, bbox_inches='tight')
        plt.show()

In [ ]:
import os, random

# Set Keras backend to TensorFlow
os.environ["KERAS_BACKEND"] = "tensorflow"

# Uncomment if you want to select a specific GPU (here GPU 0)
# os.environ["THEANO_FLAGS"]  = "device=gpu%d"%(0)

# Make only GPU 0 visible to TensorFlow/Keras
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
import matplotlib
# matplotlib.use('Tkagg')  # Optional: Use TkAgg backend for interactive plotting
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import pickle, random, sys, h5py

import keras
import keras.backend as K
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.regularizers import *
from keras.optimizers import Adam
from keras.models import model_from_json
from keras.utils.np_utils import to_categorical

# ===========================
# Define modulation classes
# ===========================
# This list represents all modulation schemes in the HisarMod2019.1 dataset
classes = ['BPSK', 'QPSK', '8PSK', '16PSK', '32PSK', '64PSK',
           '4QAM', '8QAM', '16QAM', '32QAM', '64QAM', '128QAM', '256QAM',
           '2FSK', '4FSK', '8FSK', '16FSK',
           '4PAM', '8PAM', '16PAM',
           'AM-DSB', 'AM-DSB-SC', 'AM-USB', 'AM-LSB',
           'FM', 'PM']

# Each element in the list is used as a target class label when converting labels
# to one-hot vectors using `to_categorical()` during training and evaluation.

In [ ]:
# ===========================
# Load training data
# ===========================
data1 = h5py.File('Dataset/HisarMod2019.1/Train/train.mat','r')  # Open .mat file in read mode
train = data1['data_save'][:]                                   # Extract dataset from HDF5 group 'data_save'

# Swap axes to match model input shape
# Original shape might be (num_samples, 2, 1024) or (1024, 2, num_samples)
# After swapaxes(0,2), shape becomes (num_samples, 2, 1024)
train = train.swapaxes(0,2)

# ===========================
# Load test data
# ===========================
data2 = h5py.File('Dataset/HisarMod2019.1/Test/test.mat','r')
test = data2['data_save'][:]
test = test.swapaxes(0,2)

# ===========================
# Add channel dimension
# ===========================
# Conv2D layers expect input of shape (samples, height, width, channels)
train = np.expand_dims(train, axis=3)  # Add a channel dimension: shape -> (num_samples, 2, 1024, 1)
test = np.expand_dims(test, axis=3)

In [ ]:
# ===========================
# Load and preprocess labels
# ===========================

# Load training labels from CSV file
train_labels = pd.read_csv('Dataset/HisarMod2019.1/Train/train_labels1.csv', header=None)
train_labels = np.array(train_labels)  # Convert DataFrame to NumPy array

# Convert integer labels to one-hot encoding
# This is required for categorical cross-entropy loss in Keras
train_labels = to_categorical(train_labels, num_classes=None)


# Load test labels from CSV file
test_labels = pd.read_csv('Dataset/HisarMod2019.1/Test/test_labels1.csv', header=None)
test_labels = np.array(test_labels)    # Convert DataFrame to NumPy array
test_labels = to_categorical(test_labels, num_classes=None)  # One-hot encoding


# ===========================
# Load and preprocess SNR values
# ===========================

# Load training SNR values from CSV
# Each row corresponds to the SNR of a training sample
train_snr = pd.read_csv('Dataset/HisarMod2019.1/Train/train_snr.csv', header=None)
train_snr = np.array(train_snr)  # Convert to NumPy array for easier indexing

# Load test SNR values from CSV
# Each row corresponds to the SNR of a test sample
test_snr = pd.read_csv('Dataset/HisarMod2019.1/Test/test_snr.csv', header=None)
test_snr = np.array(test_snr)    # Convert to NumPy array

In [ ]:
# ===========================
# Split dataset into training and validation sets
# ===========================

# Number of total examples in the training dataset
n_examples = train.shape[0]  # train has shape [N, 1024, 2]

# Number of samples for training and validation
n_train = int(n_examples * 0.8)  # 80% for training
n_val = int(n_examples * 0.2)    # 20% for validation

# Randomly select indices for training samples
train_idx = list(np.random.choice(range(0, n_examples), size=n_train, replace=False))

# Remaining indices will be used for validation
val_idx = list(set(range(0, n_examples)) - set(train_idx))

# Shuffle indices to ensure random ordering
np.random.shuffle(train_idx)
np.random.shuffle(val_idx)

# ===========================
# Create datasets using the indices
# ===========================

# Training data and labels
X_train = train[train_idx]         # Shape: [n_train, 1024, 2]
Y_train = train_labels[train_idx]  # Shape: [n_train, num_classes]

# Validation data and labels
X_val = train[val_idx]             # Shape: [n_val, 1024, 2]
Y_val = train_labels[val_idx]      # Shape: [n_val, num_classes]

# Test data and labels (use full test set)
X_test = test                     # Shape: [num_test_samples, 1024, 2]
Y_test = test_labels               # Shape: [num_test_samples, num_classes]

# Test SNR values corresponding to each test sample
Z_test = test_snr                  # Shape: [num_test_samples, 1] or similar

In [ ]:
# Set up some params
nb_epoch = 200     # number of epochs to train on
batch_size = 300  # training batch size

In [ ]:
# ===========================
# Initialize the CLDNN-like model
# ===========================
# `CLDNNLikeModel` is a custom model combining CNN and LSTM layers
# Input shape corresponds to (channels, time_steps) = [2, 1024]
model = CLDNNLikeModel(None, input_shape=[2,1024])

# ===========================
# Compile the model
# ===========================
# Loss: categorical_crossentropy for multi-class classification
# Metrics: accuracy to monitor performance
# Optimizer: Adam for adaptive gradient updates
model.compile(
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    optimizer='Adam'
)

# ===========================
# Visualize the model architecture
# ===========================
# `plot_model` saves a diagram of the model layers to 'model_CLDNN.png'
# `show_shapes=True` includes the shape of tensors at each layer
plot_model(model, to_file='model_CLDNN.png', show_shapes=True)
model.summary()
# After this step, you can open 'model_CLDNN.png' to inspect the architecture,
# which is useful for debugging or reporting.


In [ ]:
# ===========================
# Define file path to save best model weights
# ===========================
# During training, the best weights based on validation loss will be saved here
filepath = 'weights/CLDNN.h5'

import time

# ===========================
# Record start time for training
# ===========================
TRS_CLDNN = time.time()

# ===========================
# Train the CLDNN model
# ===========================
history = model.fit(
    X_train,           # Training input data
    Y_train,           # Training labels (one-hot encoded)
    batch_size=batch_size,  # Number of samples per gradient update
    epochs=nb_epoch,        # Total number of training epochs
    verbose=2,              # 0 = silent, 1 = progress bar, 2 = one line per epoch
    validation_data=(X_val, Y_val),  # Validation set to monitor overfitting
    callbacks=[             # Callbacks to improve training and save best weights
        # Save the model weights when validation loss improves
        keras.callbacks.ModelCheckpoint(
            filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto'
        ),
        # Reduce learning rate by factor 0.5 if validation loss plateaus
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, verbose=1, patince=5, min_lr=1e-7
        ),
        # Stop training early if validation loss doesn't improve after 50 epochs
        keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=50, verbose=1, mode='auto'
        ),
        # Optional: TensorBoard callback for visualizing training metrics
        # keras.callbacks.TensorBoard(log_dir='./logs/', histogram_freq=1, write_graph=False,
        #                             write_grads=1, write_images=False, update_freq='epoch')
    ]
)

In [ ]:
# ===========================
# Record end time after training
# ===========================
TRE_CLDNN = time.time()

# ===========================
# Calculate total training time
# ===========================
# T_CLDNN stores the elapsed time in seconds that the model took to train
T_CLDNN = TRE_CLDNN - TRS_CLDNN

# You can print T_CLDNN to see the total training duration
# print("Total training time for CLDNN: {:.2f} seconds".format(T_CLDNN))

In [ ]:
# ===========================
# Start timing the evaluation process
# ===========================
TES_CLDNN = time.time()

# ===========================
# Evaluate the trained CLDNN model on the test set
# ===========================
# X_test : test input data
# Y_test : corresponding test labels (one-hot encoded)
# verbose=1 : shows a progress bar during evaluation
# batch_size : number of samples processed at a time during evaluation
score = model.evaluate(X_test, Y_test, verbose=1, batch_size=batch_size)

# ===========================
# Print the evaluation metrics
# ===========================
# score[0] : test loss (categorical crossentropy)
# score[1] : test accuracy
print(score)

# ===========================
# This provides a quick summary of the model's performance on unseen data
# ===========================

In [ ]:
calculate_acc_cm_each_snr(Y_test, test_Y_hat, Z_test, classes, min_snr=-18) #Calculate accuracy

In [ ]:
# ===========================
# Record end time after model evaluation
# ===========================
TEE_CLDNN = time.time()

# ===========================
# Calculate total evaluation time
# ===========================
# T_CLDNN stores the elapsed time in seconds that the model took to evaluate on the test set
T_CLDNN = TEE_CLDNN - TES_CLDNN

# You can print T_CLDNN to see the total evaluation duration
# print("Total evaluation time for CLDNN: {:.2f} seconds".format(T_CLDNN))